# PICOS vs. CVXPY SDP実行速度比較

## 0. 速度計測用関数

In [ ]:
# PICOS
import picos as pic
import numpy as np

def TestPicos(solver, n_matsize, tolerance):
    # sdp object
    sdp = pic.Problem(verbosity=0)
    sdp.options["solver"] = solver
    sdp.options["*_tol"] = tolerance
    sdp.options["primals"] = False
    
    # constant matrix
    A = np.random.randn(n_matsize, n_matsize)
    # variable matrix
    P = pic.SymmetricVariable('P', (n_matsize, n_matsize))
    # objective
    sdp.set_objective("min", pic.trace(P))
    # constraint
    sdp.add_constraint(P >> 1e-08*np.eye(n_matsize))
    sdp.add_constraint(P*A + A.T*P << -np.eye(n_matsize))
    # solve
    sdp.solve()

# CVXPY
import cvxpy as cp

def TestCvxpy(solver, n_matsize, tolerance):
    # constant matrix
    A = np.random.randn(n_matsize, n_matsize)
    # variable matrix
    P = cp.Variable((n_matsize,n_matsize), symmetric=True)
    # constraint
    constraint = [P >> 1e-08*np.eye(n_matsize)]
    constraint += [P@A + A.T@P << -np.eye(n_matsize)]
    # sdp object
    prob = cp.Problem(cp.Minimize(cp.trace(P)), constraint)
    prob.solver = solver
    prob.eps = tolerance
    # solve
    prob.solve()

# others
def makePlottable(captured_timit_result):
    # timeitの出力を加工してリストで返しプロットできるようにする関数
    values = []
    stds = []
    results = captured_timit_result.stdout.split('\n')
    for res in results[:-1]:
        value, vunit, pm, std, sunit = res.split()[:5]
        value = float(value) * {"s": 1, "ms": 1e-3, "us": 1e-6, "ns": 1e-9}[vunit]
        values.append(value)
        std = float(std) * {"s": 1, "ms": 1e-3, "us": 1e-6, "ns": 1e-9}[sunit]
        stds.append(std)
    return values, stds

## 1. 行列のサイズを変化させて比較
1-1. 速度計測

In [ ]:
N = 20 # maximun size of matrix
matsizes = range(1,N+1)
tolerance = 1e-08

In [ ]:
%%capture result_picos_cvxopt
TestPicosCvxoptMatsize = lambda n: TestPicos("cvxopt", n, tolerance)
for n in matsizes:
    %timeit -r 10 -n 30 -o TestPicosCvxoptMatsize(n)

In [ ]:
%%capture result_picos_mosek
TestPicosMosekMatsize = lambda n: TestPicos("mosek", n, tolerance)
for n in matsizes:
    %timeit -r 10 -n 30  TestPicosMosekMatsize(n)

In [ ]:
%%capture result_cvxpy_cvxopt
TestCvxpyCvxoptMatsize = lambda n: TestCvxpy("cvxopt", n, tolerance)
for n in matsizes:
    %timeit -r 10 -n 30 TestCvxpyCvxoptMatsize(n)

In [ ]:
%%capture result_cvxpy_mosek
TestCvxpyMosekMatsize = lambda n: TestCvxpy("mosek", n, tolerance)
for n in matsizes:
    %timeit -r 10 -n 30 TestCvxpyMosekMatsize(n)

In [ ]:
%%capture result_cvxpy_scs
TestCvxpyScsMatsize = lambda n: TestCvxpy("scs", n, tolerance)
for n in matsizes:
    %timeit -r 10 -n 30 TestCvxpyScsMatsize(n)

1-2. プロット

In [ ]:
import matplotlib.pyplot as plt

# New Figure
fig = plt.figure()

# New Axis
ax = fig.add_subplot(111)

# picos x cvxopt
x = np.linspace(1,N,N)
values, stds = makePlottable(result_picos_cvxopt)
plt.errorbar(x, values, yerr=stds, label='PICOS x CVXOPT', marker='.')

# picos x mosek
values, stds = makePlottable(result_picos_mosek)
plt.errorbar(x, values, yerr=stds, label='PICOS x MOSEK', marker='o')

# cvxpy x cvxopt
values, stds = makePlottable(result_cvxpy_cvxopt)
plt.errorbar(x, values, yerr=stds, label='CVXPY x CVXOPT', marker='v')

# cvxpy x mosek
values, stds = makePlottable(result_cvxpy_mosek)
plt.errorbar(x, values, yerr=stds, label='CVXPY x MOSEK', marker='<')

# cvxpy x scs
values, stds = makePlottable(result_cvxpy_scs)
plt.errorbar(x, values, yerr=stds, label='CVXPY x SCS', marker='>')

# order figure
ax.set_xticks(x)
plt.rcParams["font.size"] = 10
plt.rcParams["lines.markersize"] = 6
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16);
plt.xlabel('Size of Matrix', fontsize=24)
plt.ylabel('Time [sec]', fontsize=24)

plt.savefig("matsize.jpg", bbox_inches="tight")

# 2. 許容誤差を変化させて比較
2-1. 速度計測

In [ ]:
n_matsize = 2
tolerances = [1e-04, 1e-06, 1e-08, 1e-10, 1e-12]

In [ ]:
%%capture result_picos_cvxopt
TestPicosCvxoptTolerance = lambda tol: TestPicos("cvxopt", n_matsize, tol)
for tol in tolerances:
    %timeit -r 10 -n 300 TestPicosCvxoptTolerance(tol)

In [ ]:
%%capture result_picos_mosek
tolerances_pic_msk = [1e-04, 1e-06, 1e-08]
TestPicosMosekTolerance = lambda n: TestPicos("mosek", n_matsize, tol)
for tol in tolerances_pic_msk:
    %timeit -r 10 -n 300  TestPicosMosekTolerance(tol)
# error if tolerance > 1e-08

In [ ]:
%%capture result_cvxpy_cvxopt
TestCvxpyCvxoptTolerance = lambda tol: TestCvxpy("cvxopt", n_matsize, tol)
for tol in tolerances:
    %timeit -r 10 -n 300 TestCvxpyCvxoptTolerance(tol)

In [ ]:
%%capture result_cvxpy_mosek
TestCvxpyMosekTolerance = lambda tol: TestCvxpy("mosek", n_matsize, tol)
for tol in tolerances:
    %timeit -r 10 -n 300 TestCvxpyMosekTolerance(tol)

In [ ]:
%%capture result_cvxpy_scs
TestCvxpyScsTolerance = lambda tol: TestCvxpy("scs", n_matsize, tol)
for tol in tolerances:
    %timeit -r 10 -n 300 TestCvxpyScsTolerance(tol)

2-2. プロット

In [ ]:
import matplotlib.pyplot as plt

# New Figure
fig = plt.figure()

# New Axis
ax = fig.add_subplot(111)

# picos x cvxopt
x = tolerances
values, stds = makePlottable(result_picos_cvxopt)
plt.errorbar(x, values, yerr=stds, label='PICOS x CVXOPT', marker='.')

# picos x mosek
values, stds= makePlottable(result_picos_mosek)
plt.errorbar(tolerances_pic_msk, values, yerr=stds, label='PICOS x MOSEK', marker='o')

# cvxpy x cvxopt
values, stds = makePlottable(result_cvxpy_cvxopt)
plt.errorbar(x, values, yerr=stds, label='CVXPY x CVXOPT', marker='v')

# cvxpy x mosek
values, stds = makePlottable(result_cvxpy_mosek)
plt.errorbar(x, values, yerr=stds, label='CVXPY x MOSEK', marker='<')

# cvxpy x scs
values, stds = makePlottable(result_cvxpy_scs)
plt.errorbar(x, values, yerr=stds, label='CVXPY x SCS', marker='>')

# order figure
ax.set_xticks(x)
plt.rcParams["font.size"] = 10
plt.rcParams["lines.markersize"] = 16
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');
plt.xscale('log')
plt.xlabel('Tolerance', fontsize=24)
plt.ylabel('Time [sec]', fontsize=24)
plt.xlim(1e-04, 1e-12);

plt.savefig('tolerance.jpg', bbox_inches="tight")